In [ ]:
# import pymongo
# import pandas as pd

# # --- Step 1: Extract Authors from MongoDB (OpenAlex Data) ---

# # Connect to MongoDB
# client = pymongo.MongoClient('localhost', 27017)

# # Access the 'dd-usda' database and 'publications' collection
# db = client['dd-usda']
# collection = db['publications']

# # Define the projection to include only 'authors' and 'year'
# projection = {
#     'authors': 1,
#     'year': 1,
#     '_id': 0
# }

# query = {
#     'author_american': True,
#     '$or': [
#         {
#             'aliases': {
#                 '$in': [
#                     "USDA", "USDA Census", "US Department of Agriculture",
#                     "United States Department of Agriculture", "NASS", "NASS Census of Agriculture"
#                 ]
#             }
#         },
#         {
#             'second_alias': {
#                 '$in': [
#                     "USDA", "USDA Census", "US Department of Agriculture",
#                     "United States Department of Agriculture", "NASS", "NASS Census of Agriculture"
#                 ]
#             }
#         }
#     ]
# }

# # Retrieve all documents
# cursor = collection.find(query, projection)
# documents = list(cursor)

# # Extract (author_name, year) pairs
# author_year_list = []

# for doc in documents:
#     year = doc.get('year')
#     authors = doc.get('authors', [])
    
#     # Skip documents without a year
#     if year is None:
#         continue
    
#     # Process each author
#     for author in authors:
#         name = author.get('name')
#         if name:
#             # Append the author and year to the list
#             author_year_list.append({'year': int(year), 'name': name.strip()})

# # Create DataFrame for OpenAlex authors
# openalex_authors = pd.DataFrame(author_year_list)

# # Remove duplicates
# openalex_authors.drop_duplicates(inplace=True)

# # Ensure 'year' is of integer type
# openalex_authors['year'] = openalex_authors['year'].astype(int)

In [2]:
import pandas as pd
openalex_authors = pd.read_csv("openalex_authors.csv")

# --- Step 2: Load Authors from Scopus CSV File ---
# From Sciserver, database DemocratizingData_USDA_2023, query: select distinct publication.year, '"' + publication_author.given_name + ' ' + publication_author.family_name + '"' from publication join publication_author on publication.id = publication_author.publication_id
scopus_authors = pd.read_csv('scopus_authors.csv')

# Remove duplicates
scopus_authors.drop_duplicates(inplace=True)

# Ensure 'year' is of integer type
scopus_authors['year'] = scopus_authors['year'].astype(int)

# Strip whitespace from names
scopus_authors['name'] = scopus_authors['name'].str.strip()

# --- Step 3: Determine Authors Unique to Each Source and Those in Both, by Year ---

# Combine years from both datasets
years = sorted(set(openalex_authors['year']).union(set(scopus_authors['year'])))

# Initialize a list to store the results
results = []

for year in years:
    # Get authors for the current year from OpenAlex
    oa_authors_set = set(
        openalex_authors[openalex_authors['year'] == year]['name'].str.lower().str.strip()
    )
    
    # Get authors for the current year from Scopus
    scopus_authors_set = set(
        scopus_authors[scopus_authors['year'] == year]['name'].str.lower().str.strip()
    )
    
    # Authors only in OpenAlex
    only_openalex = oa_authors_set - scopus_authors_set
    
    # Authors only in Scopus
    only_scopus = scopus_authors_set - oa_authors_set
    
    # Authors in both
    both = oa_authors_set & scopus_authors_set
    
    # Append the counts to the results list
    results.append({
        'year': year,
        'only_openalex': len(only_openalex),
        'only_scopus': len(only_scopus),
        'both': len(both)
    })

# Create a Results DataFrame
results_df = pd.DataFrame(results)

# Sort the DataFrame by year
results_df = results_df.sort_values('year')

# Display the results
print("\nAuthor Counts by Year and Source:")
print(results_df)



Author Counts by Year and Source:
    year  only_openalex  only_scopus  both
0   1991              0            3     0
1   1992              0            5     0
2   1993              0            3     0
3   1994              0           31     0
4   1995              0            9     0
5   1996              0           35     0
6   1997              0           46     0
7   1998              0           39     0
8   1999              0           48     0
9   2000              0           34     0
10  2001              0           25     0
11  2002              0           64     0
12  2003              0           92     0
13  2004              0           93     0
14  2005              0          118     0
15  2006              0          161     0
16  2007              0           90     0
17  2008              0          160     0
18  2009              0          204     0
19  2010              0          237     0
20  2011              0          183     0
21  2012           